# CGAN => conditional gan 
### this network solved the problem of randomness for the DGAN network by giving the network sense of the outputs by adding one-hot-encoding label vector to both generator and discriminator and give us the ability to control what the network generate so I used this architecture 

![CGAN](images/CGAN.png) 

In [ ]:
import tensorflow as tf 
import os 
from tensorflow.keras.layers import Dense , Conv2D ,concatenate, Conv2DTranspose , Flatten , Reshape  , BatchNormalization , Activation, Flatten ,LeakyReLU
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.utils import to_categorical
import numpy as np 
import matplotlib.pyplot as plt 
import math 
import os 
import argparse


# Generator 

In [ ]:
class Generator(tf.keras.Model) : 
    def __init__(self) : 
        super().__init__()  
        self.dense_1 = Dense(7*7*128 ) 
        self.reshape = Reshape((7,7,128))

        
        self.sequential = Sequential([
             BatchNormalization() ,
             Activation('relu') , 
             Conv2DTranspose(128 , 5 , strides = 2 , padding = 'same') , 
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(64 , 5 , strides = 2  , padding = 'same') ,
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(32 , 5   , padding = 'same') ,
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(1 , 5  , padding = 'same') , 
             Activation('sigmoid')
        ])
        
    def call(self , inputs ) :
        # we get noise vector and one hot label vector as inputs 
        X_vector , one_hot_vector = inputs
        X = concatenate([X_vector , one_hot_vector] , axis = 1) 
        X = self.reshape(self.dense_1(X ))
        return self.sequential(X)    
    
    

# Discriminator 

In [ ]:
class Discriminator(tf.keras.Model) : 
    def __init__(self) : 
        super().__init__() 
        self.label_dense = Dense(7*7*16) 
        self.reshape = Reshape((28 ,28 ,1)) 
        self.sequential = Sequential([
            LeakyReLU(.2), 
            Conv2D(32 , 5 , padding ='same' , strides = 2) , 
            LeakyReLU(.2),
            Conv2D(64 , 5 , padding ='same' , strides = 2) ,
            LeakyReLU(.2),
            Conv2D(128 , 5 , padding ='same' , strides = 2) ,
            LeakyReLU(.2),
            Conv2D(256 , 5 , padding ='same'  ) ,
            Flatten() , 
            Dense(1) , 
            Activation('sigmoid') 
        ])
        
        
    def call(self, inputs ) : 
        image ,one_hot_vector = inputs 
        # we do some sort of embedding to the label vector and reshape it to the image shape 
        lebel_embed = self.label_dense(one_hot_vector ) 
        label_embed = self.reshape(lebel_embed) 
        X = concatenate([image , label_embed]) 
        return self.sequential(X)
        
        

# Adversarial

In [ ]:
class Adversarial(tf.keras.Model) : 
    def __init__(self , generator , discriminator) : 
        super().__init__()
        self.generator = generator 
        self.discriminator = discriminator
        
    def call(self , inputs ) : 
        
        noise , labels = inputs 
        # first we pass noise and labels to generator 
        images = self.generator((noise , labels)) 
        # and we pass the generator output and labels to discriminator
        return self.discriminator((images , labels)) 

In [ ]:
def train(models, data, params):
    
    generator, discriminator, adversarial = models
    x_train, y_train = data
    batch_size, latent_size, train_steps, num_labels, model_name = params
    save_interval = 500
    # noise vector to see how the generator output evolves during training
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
    # one-hot label the noise will be conditioned to
    noise_class = np.eye(num_labels)[np.arange(0, 16) % num_labels]
    # number of elements in train dataset
    train_size = x_train.shape[0]

    print(model_name,
          "Labels for generated images: ",
          np.argmax(noise_class, axis=1))

    for i in range(train_steps):


        ##### train the discriminator for 1 batch
        # randomly pick real images from dataset
        rand_indexes = np.random.randint(0, train_size, size=batch_size)
        real_images = x_train[rand_indexes]
        # corresponding one-hot labels of real images
        real_labels = y_train[rand_indexes]
        # generate fake images from noise using generator
        # generate noise using uniform distribution
        noise = np.random.uniform(-1.0,
                                  1.0,
                                  size=[batch_size, latent_size])
        # assign random one-hot labels
        fake_labels = np.eye(num_labels)[np.random.choice(num_labels,
                                                          batch_size)]

        # generate fake images conditioned on fake labels
        fake_images = generator.predict((noise, fake_labels))
        # real + fake images = 1 batch of train data
        x = np.concatenate((real_images, fake_images))
        # real + fake one-hot labels = 1 batch of train one-hot labels
        labels = np.concatenate((real_labels, fake_labels))

        # label real and fake images
        # real images label is 1.0
        y = np.ones([2 * batch_size, 1])
        # fake images label is 0.0
        y[batch_size:, :] = 0.0
        # train discriminator network, log the loss and accuracy
        loss, acc = discriminator.train_on_batch((x, labels), y)
        log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)

        ######## train the adversarial network for 1 batch
        # since the discriminator weights are frozen in 
        # adversarial network only the generator is trained
        # generate noise using uniform distribution        
        noise = np.random.uniform(-1.0,
                                  1.0,
                                  size=[batch_size, latent_size])
        # assign random one-hot labels
        fake_labels = np.eye(num_labels)[np.random.choice(num_labels,
                                                          batch_size)]
        # label fake images as real or 1.0
        y = np.ones([batch_size, 1])
        # train the adversarial network 
        loss, acc = adversarial.train_on_batch((noise, fake_labels), y)
        log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
        print(log)
        if (i + 1) % save_interval == 0:
            # plot generator images on a periodic basis
            plot_images(generator,
                        noise_input=noise_input,
                        noise_class=noise_class,
                        show=False,
                        step=(i + 1),
                        model_name=model_name)
    
    # save the model after training the generator
    # the trained generator can be reloaded for 
    # future MNIST digit generation
    generator.save(model_name + ".h5")



In [ ]:
def plot_images(generator,
                noise_input,
                noise_class,
                show=False,
                step=0,
                model_name="gan"):
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict([noise_input, noise_class])
    print(model_name , " labels for generated images: ", np.argmax(noise_class, axis=1))
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

In [ ]:
def build_and_train_models():
    # load MNIST dataset
    (x_train, y_train), (_, _) = mnist.load_data()

    # reshape data for CNN as (28, 28, 1) and normalize
    image_size = x_train.shape[1]
    x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
    x_train = x_train.astype('float32') / 255

    num_labels = np.amax(y_train) + 1
    y_train = to_categorical(y_train)

    model_name = "cgan_mnist"
    # network parameters
    # the latent or z vector is 100-dim
    latent_size = 100
    batch_size = 64
    train_steps = 40000
    lr = 2e-4
    decay = 6e-8
    input_shape = (image_size, image_size, 1)
    label_shape = (num_labels, )

    

    discriminator = Discriminator()
    # [1] or original paper uses Adam, 
    # but discriminator converges easily with RMSprop
    optimizer = RMSprop(lr=lr, decay=decay)
    discriminator.compile(loss='binary_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])
    

   
    generator = Generator()


    # build adversarial model = generator + discriminator
    optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
    # freeze the weights of discriminator during adversarial training
    discriminator.trainable = False
    adversarial = Adversarial(generator,discriminator)
    
    adversarial.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])

    # train discriminator and adversarial networks
    models = (generator, discriminator, adversarial)
    data = (x_train, y_train)
    params = (batch_size, latent_size, train_steps, num_labels, model_name)
    train(models, data, params)


In [ ]:
build_and_train_models()